In [4]:
%%capture 
%load_ext autoreload
%autoreload 2
%reset -f

In [5]:
from functions import *

## **Import the EEG Data and set the Global Variables**

In [6]:
# chan_name = ['PO3','POz','PO4', 'O1', 'Oz', 'O2', 'PO7', 'PO8']

chan_name = ['PO3','POz','PO4', 'O1', 'Oz', 'O2']

fs, lf, hf = 250, 5, 100 #Hz

target_freq = [6.6, 7.5, 13.2, 15, 19.8, 22.5, 26.4, 30, 33, 37.5]
# target_freq = [7.5, 15, 22.5, 30, 37.5]

eeg_data = process_folder('data/Epochs/', chan_name, target_freq=target_freq, hf=hf, lf=lf, epoch_length=6, filter = False)

# keep only exp1 and 2
new_eeg_data = []
for eeg in eeg_data:
    code = int(eeg.title.split('_')[-1])
    if code >= 110 and code < 120:
        new_eeg_data.append(eeg)
eeg_data = new_eeg_data


# sort by title
eeg_data.sort(key=lambda x: (int(x.title[1]),int(x.title[-5]), int(x.title[-3:])))

In [7]:
# remove channels 
# chan_to_remove = ['Oz']

# for eeg in eeg_data:
#     eeg.remove_channels(chan_to_remove)

# for chan in chan_to_remove:
#     if chan in chan_name:
#         chan_name.remove(chan)

In [8]:
# for eeg in eeg_data:
#     eeg.filtered_signal = eeg.filtered_signal - np.mean(eeg.filtered_signal, axis=0)
    
#     mean_val = np.mean(eeg.filtered_signal, axis=0)
#     std_val = np.std(eeg.filtered_signal, axis=0)
#     scaled_signal = (eeg.filtered_signal - mean_val) / std_val

In [9]:
# for eeg in eeg_data:
#     amplitude_plot(eeg.filtered_signal, chan_name, title=(eeg.title), lim=25)

In [10]:
# psd_plot_interactive(eeg_data, chan_name, nperseg_max=50, nfft_max=50,fig_x=15,fig_y=3, xmin=lf, xlim=35, ylim=5)

In [11]:
# def rls_filter(y, f, lambda_, fs, delta):
#     # Number of channels
#     n_channels = y.shape[0]

#     # Number of filter coefficients
#     P = 2

#     # Initialize the filter coefficients
#     w_all = np.zeros([n_channels, P, y.shape[1]])

#     # Initialize the desired signal
#     x1 = np.sin(2*np.pi*f*np.arange(0,y.shape[1])/fs)
#     x2 = np.cos(2*np.pi*f*np.arange(0,y.shape[1])/fs)
#     x = np.vstack((x1,x2)) 

#     # Initialize the inverse correlation matrix
#     R = delta*np.eye(P)

#     # Initialize the output signal
#     y_hat = np.zeros_like(y)

#     # Initialize the error signal
#     e = np.zeros_like(y)

#     # Loop over channels
#     for ch in range(n_channels):
#         # Loop through the samples one by one
#         for i in range(0,y.shape[1]):
#             # Compute the adaptation gain
#             k_num = (1/lambda_)*(R@x[:,i])  
#             k_den = 1 + (1/lambda_)*(x[:,i].T@R@x[:,i])
#             k = k_num/k_den
#             k = np.reshape(k, (P,1))

#             # Error signal & update signal estimate
#             y_hat[ch, i] = np.dot(w_all[ch, :, i - 1], x[:, i])
#             e[ch, i] = y[ch, i] - y_hat[ch, i]  # Reshape y_hat[ch, i] to match the shape of y[ch, i]

#             # Update the filter coefficients
#             w_all[ch, :, i] = w_all[ch, :, i-1] + (k * e[ch, i]).flatten()

#             # Update the inverse correlation matrix
#             x_temp = np.reshape(x[:,i], (2,1))
#             R = (1/lambda_) * R - (1/lambda_)* k @ x_temp.T @ R

#     return y_hat

In [12]:
# for eeg in eeg_data:
#     eeg.filtered_signal = rls_filter(eeg.filtered_signal, 7.5, 0.999, 250, 0.001)

# for eeg in eeg_data:
#     amplitude_plot(eeg.filtered_signal, chan_name, title=(eeg.title), lim=5)

In [13]:
# psd_plot_interactive(eeg_data, chan_name, nperseg_max=50, nfft_max=50,fig_x=15,fig_y=3, xmin=lf, xlim=hf, ylim=25)

In [14]:
# for eeg in eeg_data: 
#     plot_spectrogram_and_bands(eeg, title = eeg.title, band_freqs=eeg.stimulus_frequency, nfft=30, nperseg=10)   

In [15]:
# for eeg in eeg_data: 
#     plot_spectrogram_and_bands(eeg, title = eeg.title, band_freqs=eeg.stimulus_frequency, nfft=5, nperseg=5)

## Statistical Analysis:

Statistical tests are commonly used when you have well-defined hypotheses and want to make inferences about population parameters based on sample data. They are suitable for testing specific research questions, examining relationships between variables, and providing statistical evidence for or against hypotheses. Statistical tests are often simpler to interpret and require fewer computational resources compared to machine learning algorithms.


### Research Question 1: Can we differentiate SSVEP responses based on which eye was stimulated?
Compare the SSVEP responses at 7.5Hz between the left eye stimulation 
(Lf1, R∅) and the right eye stimulation (L∅, Rf1). If there is a significant difference, it suggests that the SSVEP responses depend on which eye was stimulated. 

A suitable statistical test for this comparison could be a paired t-test or a Wilcoxon signed-rank test if the data does not meet the assumptions of normality.

$\mathrm{SNR}_{\text {Broadband }}=\frac{\sum_{\delta=1}^{harmonics}\left[N\left(\delta f_1\right)+N\left(\delta f_2\right)\right]}{\sum_{f=5 H z}^{hf H z} N(f)-\sum_{\delta=1}^{harmonics}\left[N\left(\delta f_1\right)+N\left(\delta f_2\right)\right]}$

In [16]:
from scipy import stats

def N(eeg_data, target_frequency, fs=250, nperseg=256, noverlap=128):
    # N (f) represents the power value of the signal at frequency
    f, psd = signal.welch(eeg_data, fs=fs, nperseg=nperseg, noverlap=noverlap)
    idx_target = np.argmin(np.abs(f - target_frequency))
    return psd[idx_target]

def snr_broadband(eeg_data, f1, f2, hf, harmonics): 
    numerator = sum([N(eeg_data, delta * f1) + N(eeg_data, delta * f2) for delta in range(1, harmonics + 1)])
    denominator = sum([N(eeg_data, f) for f in range(5, hf + 1)]) - numerator
    return numerator / denominator

def calculate_snr(eeg_data, target_frequency, fs=250, nperseg=256, noverlap=128, object=True):
    if object: eeg_data = eeg_data.filtered_signal
    # Calculate SNR for all channels
    snr = []
    # Calculate SNR per channel
    for eeg in eeg_data:
        snr.append(snr_broadband(eeg, target_frequency[0], target_frequency[1], hf=100, harmonics=5))
    # return snr for all channels 
    return np.array(snr)

def get_hemisphere_data(data, hemisphere, channels):
    left_hemisphere, midline_hemisphere, right_hemisphere = chan_hemisphere(channels)
    idx = []
    if hemisphere == 'left':
        idx = [channels.index(ch) for ch in left_hemisphere]
    elif hemisphere == 'right':
        idx = [channels.index(ch) for ch in right_hemisphere]
    elif hemisphere == 'midline':
        idx = [channels.index(ch) for ch in midline_hemisphere]
    else: return None

    return data[idx]

import copy

def stimulated_and_not(eeg, stimulated_eye):
    eeg_not_stimulated = copy.deepcopy(eeg)
    eeg_stimulated = copy.deepcopy(eeg)

    if stimulated_eye == 'left':
        eeg_stimulated.filtered_signal = get_hemisphere_data(eeg.filtered_signal, 'right', eeg.chan_name)
        eeg_not_stimulated.filtered_signal = get_hemisphere_data(eeg.filtered_signal, 'left', eeg.chan_name)

    elif stimulated_eye == 'right':
        eeg_stimulated.filtered_signal = get_hemisphere_data(eeg.filtered_signal, 'left', eeg.chan_name)
        eeg_not_stimulated.filtered_signal = get_hemisphere_data(eeg.filtered_signal, 'right', eeg.chan_name)
    
    elif stimulated_eye == 'both':
        eeg_stimulated.filtered_signal = get_hemisphere_data(eeg.filtered_signal, 'left', eeg.chan_name)
        eeg_stimulated.add_signal(get_hemisphere_data(eeg.filtered_signal, 'right', eeg.chan_name))
        eeg_not_stimulated = None

    return eeg_stimulated, eeg_not_stimulated

It seems that your data suggests differences in the SNR between the left eye, both eyes, and the right eye stimulation. Here's a brief summary of the analyses:

The paired t-test results show that there is a significant difference between the left and right eye stimulation for Participant 1 and Participant 3 (p < 0.05). However, for the remaining participants, there's no significant difference found between the left and right eye stimulation (p > 0.05).

The one-way repeated measures ANOVA showed a significant difference between the three groups (Left eye, Both eyes, Right eye) only for Participant 3 (p < 0.05). For the remaining participants, no significant difference was found (p > 0.05).

These results suggest that for the majority of participants, there is no significant difference in the SSVEP responses based on the eye that was stimulated or whether both eyes were stimulated. However, for Participant 3, the results suggest a significant difference, indicating that the SSVEP responses may depend on which eye was stimulated or whether both eyes were stimulated.

As a next step, you might want to conduct a post-hoc analysis for the one-way repeated measures ANOVA of Participant 3 to determine exactly where the differences lie. You can use a post-hoc test with a Bonferroni correction to adjust for multiple comparisons.

Furthermore, to get a clear picture across all participants, you could use a mixed model ANOVA where the fixed factor is the type of stimulation (Left eye, Both eyes, Right eye) and the random factor is the participant ID.

Finally, you might want to perform a meta-analysis to combine the p-values from the different participants into a single p-value. This can give you a single measure of the overall effect. There are different methods to combine p-values, such as Fisher's method, Stouffer's method, or others. The choice of method may depend on your specific hypothesis and data distribution.

In [17]:
def statistical_analysis(eeg_data, target_freq, compare, stimulated = True):
    # Create 
    all_compare_1 = EEG_Data(None,title = f'All_stimulated', chan_name = eeg_data[0].chan_name, stimulus_frequency= target_freq)
    all_compare_2 = EEG_Data(None,title = f'All_not_stimulated', chan_name = eeg_data[0].chan_name, stimulus_frequency= target_freq)

    for i in range(0,len(eeg_data)-2,3):
        left_eye_75hz, both_eyes_75hz, right_eye_75hz = eeg_data[i:i+3]

        # init empty EEG_Data objects
        compare_1 = EEG_Data(None,title = f'P{left_eye_75hz.title[1]}_stimulated', chan_name = left_eye_75hz.chan_name, stimulus_frequency= target_freq)
        compare_2 = EEG_Data(None,title = f'P{left_eye_75hz.title[1]}_not_stimulated', chan_name = left_eye_75hz.chan_name, stimulus_frequency= target_freq)
        
        if stimulated:
            if 'left' in compare:
                compare_1.add_signal(stimulated_and_not(left_eye_75hz, 'left')[0])
                compare_2.add_signal(stimulated_and_not(left_eye_75hz, 'left')[1])
            if 'right' in compare:
                compare_1.add_signal(stimulated_and_not(right_eye_75hz, 'right')[0])
                compare_2.add_signal(stimulated_and_not(right_eye_75hz, 'right')[1])
            if 'both' in compare:
                compare_1.add_signal(stimulated_and_not(both_eyes_75hz, 'both')[0])
        else:
            if 'left' in compare:
                compare_1.add_signal(left_eye_75hz)
            if 'right' in compare:
                compare_2.add_signal(right_eye_75hz)
            if 'both' in compare:
                if compare_1.filtered_signal is None:
                    compare_1.add_signal(both_eyes_75hz)
                elif compare_2.filtered_signal is None:
                    compare_2.add_signal(both_eyes_75hz)
                else:
                    print('Error: both eyes already added to both groups')
                    return

        # Calculate SNR for left and right hemispheres
        compare_1_snr = calculate_snr(compare_1, target_freq, object=True)
        compare_2_snr = calculate_snr(compare_2, target_freq, object=True)  
        statistic, pvalue = stats.wilcoxon(compare_1_snr, compare_2_snr)

        print(f"{left_eye_75hz.title[1]} & ",end='')
        print(*[f"{round(x, 2):.2f}" for x in compare_1_snr], sep=', ', end=' & ')
        print(*[f"{round(x, 2):.2f}" for x in compare_2_snr], sep=', ', end=' & ')
        print(f" {round(pvalue, 2)} \\\\")

        # Add to all_stimulated and all_not_stimulated
        all_compare_1.add_signal(compare_1, stack=True)
        all_compare_2.add_signal(compare_2, stack=True)

    # Calculate SNR for all_stimulated and all_not_stimulated
    all_compare_1_snr = calculate_snr(all_compare_1, target_freq, object=True)
    all_compare_2_snr = calculate_snr(all_compare_2, target_freq, object=True)
    statistic, pvalue = stats.wilcoxon(all_compare_1_snr, all_compare_2_snr)
    print("all & ",end="") 
    print(*[f"{round(x, 2):.2f}" for x in all_compare_1_snr], sep=', ', end=' & ')
    print(*[f"{round(x, 2):.2f}" for x in all_compare_2_snr], sep=', ', end=' & ')
    print(f"{round(pvalue, 2)} \\\\")

target_freq = [6.6, 7.5]
compare = ['left', 'right']
stimulated = False
print(compare, stimulated) 
statistical_analysis(eeg_data, target_freq, compare, stimulated=stimulated )

['left', 'right'] False
1 & 0.45, 0.22, 0.35, 0.30, 0.36, 0.23 & 0.42, 0.28, 0.31, 0.28, 0.36, 0.27 &  1.0 \\
1 & 0.38, 0.23, 0.34, 0.27, 0.16, 0.22 & 0.44, 0.28, 0.42, 0.32, 0.21, 0.30 &  0.03 \\
2 & 0.35, 0.28, 0.34, 0.33, 0.33, 0.34 & 0.35, 0.31, 0.35, 0.30, 0.36, 0.37 &  0.22 \\
2 & 0.47, 0.29, 0.40, 0.31, 0.37, 0.38 & 0.40, 0.31, 0.36, 0.29, 0.35, 0.35 &  0.16 \\
3 & 0.31, 0.26, 0.29, 0.36, 0.31, 0.30 & 0.28, 0.26, 0.27, 0.28, 0.29, 0.31 &  0.16 \\
4 & 0.34, 0.40, 0.40, 0.28, 0.40, 0.29 & 0.36, 0.37, 0.37, 0.27, 0.35, 0.32 &  0.31 \\
5 & 0.29, 0.30, 0.30, 0.41, 0.17, 0.36 & 0.26, 0.32, 0.29, 0.37, 0.16, 0.34 &  0.22 \\
5 & 0.36, 0.38, 0.35, 0.45, 0.22, 0.42 & 0.31, 0.38, 0.33, 0.36, 0.23, 0.42 &  0.44 \\
6 & 0.45, 0.47, 0.36, 0.35, 0.42, 0.34 & 0.46, 0.44, 0.37, 0.34, 0.38, 0.31 &  0.31 \\
7 & 0.33, 0.38, 0.28, 0.30, 0.32, 0.35 & 0.33, 0.38, 0.27, 0.26, 0.28, 0.36 &  0.44 \\
8 & 0.28, 0.23, 0.29, 0.29, 0.37, 0.21 & 0.30, 0.20, 0.30, 0.25, 0.33, 0.25 &  0.84 \\
all & 0.34, 0.29, 0.